In [1]:
import re

cls_labels ={0: 'Knife_Handle',
 1: 'Metzenbaum',
 2: 'Needle',
 3: 'Needle_Holder',
 4: 'Suture_Scissor',
 5: 'Smooth_Forceps',
 6: 'Teeth_Forceps',
 7: 'mosquito',
 8: 'thread'}

code_to_names: {'A':'Knife_Handle',
 'B': 'Metzenbaum',
 'C': 'Needle',
 'D': 'Needle_Holder',
 'E': 'Suture_Scissor',
 'F': 'Smooth_Forceps',
 'G': 'Teeth_Forceps',
 'H': 'mosquito',
 'I': 'thread'}

names_to_code = {'Knife_Handle':'A',
 'Metzenbaum':'B',
 'Needle':'C',
 'Needle_Holder':'D',
 'Suture_Scissor':'E',
 'Smooth_Forceps':'F',
 'Teeth_Forceps':'G',
 'mosquito':'H',
 'thread':'I'
}

# code_to_surgery_name = {'AHBFDE':'Skin Surgery',
#  'AHFBDE':'Open Surgery'
# }


def code_to_surgery_name(code):
    # 定義模式
    patterns = {
        'Surgery_1': r'^AHGBDE$',                         # 完全匹配 AHGBDE
        'Surgery_2': r'^AHBGE$',                          # 完全匹配 AHBGE
        'Surgery_3': r'^AH{2,4}BG{1,2}DE$',               # H 出現 2 到 4 次, G 出現 1 到 2 次
        'Surgery_4': r'^AH{4,6}G{2,4}BDE$'                # H 出現 4 到 6 次, G 出現 2 到 4 次
    }
    
    # 逐個模式進行匹配
    for surgery, pattern in patterns.items():
        if re.match(pattern, code):
            return surgery

    return "UNKNOWN"

# # 測試
# test_codes = ['AHGBDE', 'AHBGE', 'AHHHHBGGBDE', 'AHHHHHHGGGGBDE', 'AHHHBGGBDE']
# for code in test_codes:
#     print(f"{code}: {classify_surgery(code)}")

In [2]:
model_path = '/home/mycena/knife_imgs/models/20241104_1931/knife/runs/segment/train3/weights/best.pt'
output_video_name = 'real_time_output.mp4'

In [3]:
def generate_code(data, names_to_code):
    code = ''
    for item in data:
        # item[4] 是工具名称
        name = item[4]
        if name in names_to_code:
            code += names_to_code[name]
    return code


## Session 2
「我的result裡面有一些instance會有xy座標太相近以致會有重疊的情況，

  我希望能以conf為依據，來篩掉conf比較小的那個結果」

In [4]:
# 非極大值抑制函數
def non_max_suppression(boxes, conf_threshold=0.5, iou_threshold=0.4):
    if len(boxes) == 0:
        return []

    # 先將 boxes 按照置信度排序
    boxes = sorted(boxes, key=lambda x: x['confidence'], reverse=True)
    selected_boxes = []

    while boxes:
        # 取出置信度最高的框
        current_box = boxes.pop(0)
        selected_boxes.append(current_box)

        # 計算與其他框的 IoU 並篩選
        boxes = [
            box for box in boxes
            if iou(current_box['bbox'], box['bbox']) < iou_threshold
        ]

    return selected_boxes

# IoU 計算函數
def iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    
    # 計算交集區域的面積
    interArea = max(0, xB - xA) * max(0, yB - yA)
    if interArea == 0:
        return 0.0

    # 計算各自框的面積
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])

    # 計算 IoU
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

In [5]:
# 僅使用非極大值抑制來篩選框

import cv2
from ultralytics import YOLO
import time
import copy

cap = cv2.VideoCapture(4)
model = YOLO(model_path)
order_status = "UNKNOWN"

# 設置錄製的編碼格式和參數（這裡使用 XVID 編碼器）
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = None  # 初始化錄製變量
recording = False  # 是否正在錄製的標誌

print("按下 'a' 開始錄製，'q' 停止錄製並退出")

dete_hist = []


while cap.isOpened():
    ret, org_frame = cap.read()
    if not ret: break
    
    frame = copy.deepcopy(org_frame)
    height, width = frame.shape[:2]

    results = model(frame, iou=0.7)

    # 取得偵測框並將其整理成適合 NMS 的格式
    boxes = []
        
    # 清空舊的偵測結果
    detections = []

    for result in results:
        for box in result.boxes:
            # 取得邊框座標與標籤
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # 取得框座標
            label = result.names[int(box.cls[0])]
            confidence = box.conf[0]
            boxes.append({
                'bbox': (x1, y1, x2, y2),
                'confidence': confidence,
                'label': label
            })

    # 使用非極大值抑制來篩選框
    filtered_boxes = non_max_suppression(boxes)
    
            
    for box in filtered_boxes:
        x1, y1, x2, y2 = box['bbox']
        confidence = box['confidence']
        label = box['label']

        detections.append((x1, y1, x2, y2, label, confidence))

        # 繪製邊框與標籤
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # 綠色邊框
        cv2.putText(frame, f'{label} {confidence:.2f}', (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0, 255, 0), 1)

    # 檢查排序是否正確        
    ## 按 x 座標排序
    detections.sort(key=lambda x: x[0])

        
    detect_status = generate_code(detections, names_to_code)
    dete_hist.append(detect_status)
    order_status = code_to_surgery_name(detect_status)
    
    y_offset = 20
    # 顯示排序狀態（手術名稱）
    cv2.putText(frame, f'Sequence Status:', (30, int(height*2//21)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
    cv2.putText(frame, f'{order_status}', (30, int(height*2//21)+y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    # 顯示結果
    cv2.imshow("YOLOv11 Real-Time Detection", frame)
    
    k = cv2.waitKey(33)
    if k == ord('q'):
        break
    elif k == ord('a') :
        out = cv2.VideoWriter(output_video_name, fourcc, 20.0, (frame.shape[1], frame.shape[0]))
        recording = True
        print("開始錄製...")
    if recording:
        out.write(frame)


cap.release()
if recording:
    out.release()
cv2.destroyAllWindows()

/home/mycena/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


按下 'a' 開始錄製，'q' 停止錄製並退出

0: 480x640 1 Metzenbaum, 6 Needle_Holders, 1 Suture_Scissor, 4 Teeth_Forcepss, 2 mosquitos, 622.2ms
Speed: 2.0ms preprocess, 622.2ms inference, 6.0ms postprocess per image at shape (1, 3, 480, 640)



0: 480x640 1 Metzenbaum, 6 Needle_Holders, 1 Suture_Scissor, 5 Teeth_Forcepss, 1 mosquito, 746.1ms
Speed: 1.5ms preprocess, 746.1ms inference, 5.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Metzenbaum, 6 Needle_Holders, 1 Suture_Scissor, 5 Teeth_Forcepss, 1 mosquito, 707.1ms
Speed: 2.0ms preprocess, 707.1ms inference, 6.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Metzenbaum, 6 Needle_Holders, 1 Suture_Scissor, 5 Teeth_Forcepss, 2 mosquitos, 678.3ms
Speed: 5.9ms preprocess, 678.3ms inference, 6.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Metzenbaum, 6 Needle_Holders, 1 Suture_Scissor, 5 Teeth_Forcepss, 1 mosquito, 689.3ms
Speed: 2.7ms preprocess, 689.3ms inference, 5.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 Metzenbaums, 6 Needle_Holders, 1 Suture_Scissor, 6 Teeth_Forcepss, 1 mosquito, 689.7ms
Speed: 1.9ms preprocess, 689.7ms inference, 6.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x6


0: 480x640 1 Knife_Handle, 3 Metzenbaums, 4 Needle_Holders, 1 Suture_Scissor, 4 Teeth_Forcepss, 2 mosquitos, 658.1ms
Speed: 2.1ms preprocess, 658.1ms inference, 6.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Knife_Handle, 4 Metzenbaums, 2 Needle_Holders, 1 Suture_Scissor, 4 Teeth_Forcepss, 4 mosquitos, 637.7ms
Speed: 1.4ms preprocess, 637.7ms inference, 6.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Knife_Handle, 3 Metzenbaums, 2 Needle_Holders, 1 Suture_Scissor, 4 Teeth_Forcepss, 5 mosquitos, 609.0ms
Speed: 1.3ms preprocess, 609.0ms inference, 6.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Knife_Handle, 3 Metzenbaums, 2 Needle_Holders, 1 Suture_Scissor, 4 Teeth_Forcepss, 5 mosquitos, 613.0ms
Speed: 1.0ms preprocess, 613.0ms inference, 10.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Knife_Handle, 1 Metzenbaum, 4 Needle_Holders, 1 Suture_Scissor, 4 Teeth_Forcepss, 3 mosquitos, 698.0ms
Speed: 3.8ms prepr

Speed: 1.3ms preprocess, 626.5ms inference, 6.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Knife_Handle, 1 Metzenbaum, 1 Needle_Holder, 2 Suture_Scissors, 3 Teeth_Forcepss, 6 mosquitos, 624.5ms
Speed: 1.5ms preprocess, 624.5ms inference, 6.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Knife_Handle, 1 Metzenbaum, 1 Needle_Holder, 2 Suture_Scissors, 3 Teeth_Forcepss, 6 mosquitos, 647.4ms
Speed: 1.7ms preprocess, 647.4ms inference, 6.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Knife_Handle, 1 Metzenbaum, 1 Needle_Holder, 2 Suture_Scissors, 4 Teeth_Forcepss, 5 mosquitos, 627.1ms
Speed: 1.3ms preprocess, 627.1ms inference, 6.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Knife_Handle, 1 Metzenbaum, 2 Needle_Holders, 2 Suture_Scissors, 3 Teeth_Forcepss, 5 mosquitos, 619.9ms
Speed: 1.2ms preprocess, 619.9ms inference, 6.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Knife_Handle, 1 Metzenbaum,

...

Speed: 1.6ms preprocess, 641.7ms inference, 6.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 642.4ms
Speed: 1.2ms preprocess, 642.4ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Knife_Handle, 1 Metzenbaum, 1 Needle_Holder, 1 Smooth_Forceps, 6 Teeth_Forcepss, 667.7ms
Speed: 1.9ms preprocess, 667.7ms inference, 5.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Suture_Scissor, 1 Teeth_Forceps, 638.2ms
Speed: 1.7ms preprocess, 638.2ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Knife_Handle, 2 Metzenbaums, 1 Needle_Holder, 4 Teeth_Forcepss, 2 mosquitos, 636.5ms
Speed: 2.6ms preprocess, 636.5ms inference, 5.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Knife_Handle, 4 Metzenbaums, 2 Needle_Holders, 1 Suture_Scissor, 4 Teeth_Forcepss, 3 mosquitos, 649.9ms
Speed: 1.5ms preprocess, 649.9ms inference, 7.1ms postprocess per image at shape (1, 3, 480, 640)


Speed: 1.8ms preprocess, 659.2ms inference, 6.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Knife_Handle, 3 Metzenbaums, 1 Needle_Holder, 1 Suture_Scissor, 4 Teeth_Forcepss, 3 mosquitos, 657.0ms
Speed: 1.2ms preprocess, 657.0ms inference, 6.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Knife_Handle, 3 Metzenbaums, 1 Needle_Holder, 2 Suture_Scissors, 4 Teeth_Forcepss, 4 mosquitos, 655.2ms
Speed: 1.9ms preprocess, 655.2ms inference, 7.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Knife_Handle, 4 Metzenbaums, 1 Needle_Holder, 1 Suture_Scissor, 4 Teeth_Forcepss, 4 mosquitos, 653.2ms
Speed: 2.2ms preprocess, 653.2ms inference, 7.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Knife_Handle, 4 Metzenbaums, 2 Needle_Holders, 1 Suture_Scissor, 4 Teeth_Forcepss, 3 mosquitos, 652.5ms
Speed: 1.8ms preprocess, 652.5ms inference, 7.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Knife_Handle, 3 Metzenbaum


0: 480x640 1 Knife_Handle, 4 Metzenbaums, 1 Needle_Holder, 1 Suture_Scissor, 4 Teeth_Forcepss, 4 mosquitos, 629.1ms
Speed: 3.6ms preprocess, 629.1ms inference, 7.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Knife_Handle, 4 Metzenbaums, 1 Needle_Holder, 1 Suture_Scissor, 4 Teeth_Forcepss, 4 mosquitos, 635.6ms
Speed: 1.2ms preprocess, 635.6ms inference, 7.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Knife_Handle, 3 Metzenbaums, 1 Needle_Holder, 1 Suture_Scissor, 4 Teeth_Forcepss, 4 mosquitos, 642.3ms
Speed: 1.3ms preprocess, 642.3ms inference, 6.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Knife_Handle, 3 Metzenbaums, 1 Needle_Holder, 1 Suture_Scissor, 4 Teeth_Forcepss, 4 mosquitos, 664.3ms
Speed: 1.9ms preprocess, 664.3ms inference, 6.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Knife_Handle, 4 Metzenbaums, 1 Needle_Holder, 1 Suture_Scissor, 4 Teeth_Forcepss, 4 mosquitos, 660.8ms
Speed: 1.5ms preprocess

Speed: 1.7ms preprocess, 630.8ms inference, 7.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Knife_Handle, 3 Metzenbaums, 2 Needle_Holders, 1 Suture_Scissor, 4 Teeth_Forcepss, 3 mosquitos, 1 thread, 631.7ms
Speed: 1.8ms preprocess, 631.7ms inference, 7.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Knife_Handle, 3 Metzenbaums, 2 Needle_Holders, 1 Suture_Scissor, 4 Teeth_Forcepss, 4 mosquitos, 1 thread, 705.6ms
Speed: 6.8ms preprocess, 705.6ms inference, 6.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Knife_Handle, 3 Metzenbaums, 2 Needle_Holders, 2 Suture_Scissors, 4 Teeth_Forcepss, 4 mosquitos, 1 thread, 661.5ms
Speed: 1.4ms preprocess, 661.5ms inference, 8.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Knife_Handle, 3 Metzenbaums, 2 Needle_Holders, 1 Suture_Scissor, 4 Teeth_Forcepss, 4 mosquitos, 1 thread, 653.2ms
Speed: 1.2ms preprocess, 653.2ms inference, 7.7ms postprocess per image at shape (1, 3, 480, 64

Speed: 1.5ms preprocess, 651.9ms inference, 6.7ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Knife_Handle, 4 Metzenbaums, 1 Needle_Holder, 1 Suture_Scissor, 4 Teeth_Forcepss, 4 mosquitos, 644.0ms
Speed: 1.5ms preprocess, 644.0ms inference, 7.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Knife_Handle, 5 Metzenbaums, 1 Needle_Holder, 1 Suture_Scissor, 4 Teeth_Forcepss, 3 mosquitos, 653.2ms
Speed: 1.2ms preprocess, 653.2ms inference, 7.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Knife_Handle, 4 Metzenbaums, 1 Needle_Holder, 1 Suture_Scissor, 4 Teeth_Forcepss, 4 mosquitos, 648.3ms
Speed: 1.2ms preprocess, 648.3ms inference, 7.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Knife_Handle, 4 Metzenbaums, 1 Needle_Holder, 2 Suture_Scissors, 4 Teeth_Forcepss, 3 mosquitos, 644.8ms
Speed: 2.4ms preprocess, 644.8ms inference, 7.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 Knife_Handle, 4 Metzenbaums

In [ ]:
# 用來檢查到底偵測到的東西是什麼排列
# mylist = list(dict.fromkeys(dete_hist))
# mylist